# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
import sys

from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier

import pickle

In [2]:
import os
os.getcwd()

'/home/workspace'

In [3]:
# load data from database
engine = create_engine("sqlite:///ETL.db")
df = pd.read_sql_table('df',engine)
X = df['message']
y = df.iloc[:,5:]

### 2. Write a tokenization function to process your text data

In [4]:
import nltk
import re
from nltk.tokenize import word_tokenize, WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer

In [5]:
def tokenize(text):
    token_list = WhitespaceTokenizer().tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    final_tokens = []
    for i in token_list:
        i = lemmatizer.lemmatize(i).lower().strip('!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~')
        i = re.sub(r'\[[^.,;:]]*\]', '', i)
        
        # add token to compiled list if not empty
        if i != '':
            final_tokens.append(i)
        
    return final_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
pipeline = Pipeline([('vect', CountVectorizer(tokenizer=tokenize)),
                      ('tfidf', TfidfTransformer()),
                      ('clf', MultiOutputClassifier(RandomForestClassifier()))])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [8]:
y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
import random
from sklearn.model_selection import train_test_split
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

random.seed(0)
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2)

model = pipeline
model.fit(X_train, y_train)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [10]:
from sklearn.metrics import classification_report, accuracy_score

In [11]:
y_pred = pipeline.predict(X_test)

In [12]:
print(classification_report(y_test.iloc[:,1:].values, np.array([x[1:] for x in y_pred]), target_names=df.columns[5:]))

                        precision    recall  f1-score   support

               related       0.85      0.41      0.55       906
               request       0.00      0.00      0.00        33
                 offer       0.75      0.55      0.64      2178
           aid_related       0.79      0.05      0.09       408
          medical_help       0.61      0.08      0.14       244
      medical_products       0.67      0.03      0.05       157
     search_and_rescue       0.50      0.02      0.04        91
              security       0.53      0.06      0.11       156
              military       0.00      0.00      0.00         0
           child_alone       0.79      0.17      0.28       318
                 water       0.85      0.30      0.44       585
                  food       0.85      0.22      0.35       459
               shelter       0.47      0.09      0.14        82
              clothing       0.75      0.02      0.05       127
                 money       1.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 35, does not match size of target_names, 36
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [13]:
parameters = {'vect__ngram_range':[(1,2),(2,2)],'clf__estimator__n_estimators':[10,25]}

cv = GridSearchCV(pipeline, parameters)
cv.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'vect__ngram_range': [(1, 2), (2, 2)], 'clf__estimator__n_estimators': [10, 25]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [22]:
cv.best_params_

{'clf__estimator__n_estimators': 25, 'vect__ngram_range': (1, 2)}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [14]:
y_pred = cv.predict(X_test)
print(classification_report(y_test.iloc[:,1:].values, np.array([x[1:] for x in y_pred]), target_names=df.columns[5:]))

                        precision    recall  f1-score   support

               related       0.86      0.45      0.59       906
               request       0.00      0.00      0.00        33
                 offer       0.77      0.58      0.66      2178
           aid_related       0.72      0.06      0.10       408
          medical_help       0.50      0.05      0.08       244
      medical_products       0.76      0.08      0.15       157
     search_and_rescue       1.00      0.01      0.02        91
              security       0.43      0.04      0.07       156
              military       0.00      0.00      0.00         0
           child_alone       0.79      0.37      0.51       318
                 water       0.85      0.51      0.63       585
                  food       0.87      0.25      0.39       459
               shelter       0.53      0.11      0.18        82
              clothing       0.50      0.02      0.05       127
                 money       0.67      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 35, does not match size of target_names, 36
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [18]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier

In [19]:
pipeline_kneighbors = Pipeline([('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', KNeighborsClassifier())])
    
pipeline_ada = Pipeline([('vect',CountVectorizer(tokenizer = tokenize)),
        ('tfidf',TfidfTransformer()),
        ('clf', MultiOutputClassifier(AdaBoostClassifier(n_estimators=50, learning_rate = 1)))
])

In [20]:
pipeline_kneighbors.fit(X_train, y_train)
y_pred = pipeline_kneighbors.predict(X_test)
print(classification_report(y_test.iloc[:,1:].values, np.array([x[1:] for x in y_pred]), target_names=df.columns[5:]))

                        precision    recall  f1-score   support

               related       0.80      0.34      0.48       906
               request       0.00      0.00      0.00        33
                 offer       0.72      0.30      0.42      2178
           aid_related       0.33      0.04      0.08       408
          medical_help       0.22      0.06      0.09       244
      medical_products       0.40      0.01      0.02       157
     search_and_rescue       0.00      0.00      0.00        91
              security       0.46      0.04      0.07       156
              military       0.00      0.00      0.00         0
           child_alone       0.51      0.12      0.19       318
                 water       0.58      0.21      0.30       585
                  food       0.47      0.12      0.19       459
               shelter       0.42      0.06      0.11        82
              clothing       0.67      0.05      0.09       127
                 money       1.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 35, does not match size of target_names, 36
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [21]:
pipeline_ada.fit(X_train, y_train)
y_pred = pipeline_ada.predict(X_test)
print(classification_report(y_test.iloc[:,1:].values, np.array([x[1:] for x in y_pred]), target_names=df.columns[5:]))

                        precision    recall  f1-score   support

               related       0.74      0.54      0.62       906
               request       0.00      0.00      0.00        33
                 offer       0.76      0.61      0.68      2178
           aid_related       0.64      0.22      0.33       408
          medical_help       0.49      0.23      0.32       244
      medical_products       0.56      0.18      0.28       157
     search_and_rescue       0.29      0.04      0.08        91
              security       0.55      0.36      0.44       156
              military       0.00      0.00      0.00         0
           child_alone       0.69      0.65      0.67       318
                 water       0.79      0.66      0.72       585
                  food       0.75      0.52      0.61       459
               shelter       0.59      0.32      0.41        82
              clothing       0.56      0.25      0.35       127
                 money       0.52      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 35, does not match size of target_names, 36
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 9. Export your model as a pickle file

In [24]:
from sklearn.externals import joblib
joblib.dump(cv.best_params_,'cv_model.pkl')

['cv_model.pkl']

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.